In [ ]:
import kagglehub
path = kagglehub.dataset_download("shuvoalok/ck-dataset")
print("Path to dataset files:", path)


100%|██████████| 1.80M/1.80M [00:00<00:00, 147MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/shuvoalok/ck-dataset/versions/1


In [ ]:
#install and setup prerequirsets
!pip install kagglehub matplotlib numpy pandas tensorflow keras tqdm
#importing libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tqdm import tqdm
import cv2

#download the dataset
# Step 2: Download the Dataset
import kagglehub
path = kagglehub.dataset_download("shuvoalok/ck-dataset")
print("Path to dataset files:", path)

# List all files and directories
print("Files in dataset directory:", os.listdir(path))

# Dataset base path
dataset_path = "/root/.cache/kagglehub/datasets/shuvoalok/ck-dataset/versions/1"

# Step 1: Load Dataset
def load_dataset(data_dir, target_emotion="anger", img_size=(128, 128)):
    images = []
    labels = []

    for emotion in os.listdir(data_dir):
        emotion_path = os.path.join(data_dir, emotion)
        if os.path.isdir(emotion_path):
            label = 1 if emotion == target_emotion else 0  # Label 1 for target emotion (anger), 0 otherwise
            for img_name in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    images.append(img)
                    labels.append(label)

    images = np.array(images).astype("float32") / 255.0  # Normalize to [0, 1]
    images = np.expand_dims(images, axis=-1)  # Add channel dimension
    labels = np.array(labels)
    return images, labels

# Define dataset directory and load data
data_dir = "/root/.cache/kagglehub/datasets/shuvoalok/ck-dataset/versions/1"
images, labels = load_dataset(data_dir, target_emotion="anger")
angry_images = images[labels == 1]
print(f"Angry Images Shape: {angry_images.shape}")

# Step 2: Build GAN Models
# Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128 * 32 * 32, activation="relu", input_dim=100),
        layers.Reshape((32, 32, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same", activation="relu"),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same", activation="relu"),
        layers.Conv2D(1, (7, 7), activation="tanh", padding="same")
    ])
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", input_shape=(128, 128, 1)),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.4),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

# Compile Models
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Combined GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    model.compile(optimizer="adam", loss="binary_crossentropy")
    return model

gan = build_gan(generator, discriminator)

# Train GAN
def train_gan(generator, discriminator, gan, real_data, epochs=500, batch_size=8):
    half_batch = batch_size // 2
    print(f"Training on data shape: {real_data.shape}")

    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, real_data.shape[0], half_batch)
        real_imgs = real_data[idx]

        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_imgs = generator.predict(noise)

        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss[0]} | G Loss: {g_loss}")


        # Train the GAN
train_gan(generator, discriminator, gan, angry_images, epochs=500, batch_size=8)

# Save the models
generator.save("anger_generator_model.h5")
discriminator.save("anger_discriminator_model.h5")
gan.save("anger_gan_model.h5")












Path to dataset files: /root/.cache/kagglehub/datasets/shuvoalok/ck-dataset/versions/1
Files in dataset directory: ['anger', 'fear', 'surprise', 'sadness', 'contempt', 'happy', 'disgust']
Angry Images Shape: (135, 128, 128, 1)
Training on data shape: (135, 128, 128, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Epoch 1/500 | D Loss: 0.6711550951004028 | G Loss: [array(0.6786515, dtype=float32), array(0.6786515, dtype=float32), array(0.5, dtype=float32)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━

In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import files

# Load the trained generator model
generator = load_model("anger_generator_model.h5")

# Define the image size used during GAN training
img_size = 128

# Function to preprocess an uploaded image
def preprocess_image(image_path, img_size=128):
    """
    Preprocess the uploaded image for the GAN.

    Parameters:
        image_path (str): Path to the uploaded image.
        img_size (int): Target size for resizing the image.

    Returns:
        np.array: Preprocessed image normalized to [-1, 1].
    """
    try:
        img = Image.open(image_path).convert("RGB")
        img = img.resize((img_size, img_size))
        img_array = np.asarray(img).astype("float32")
        img_array = img_array / 127.5 - 1.0  # Normalize to [-1, 1]
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array
    except Exception as e:
        print(f"Error in preprocessing image: {e}")
        return None

# Function to generate and visualize the angry face
def test_gan_model(generator):
    """
    Test the GAN model by generating an angry face from an uploaded image.

    Parameters:
        generator (tf.keras.Model): Pre-trained generator model.
    """
    print("Please upload a face image (any format like .jpg, .png, etc.)")
    uploaded = files.upload()

    for image_name in uploaded.keys():
        # Preprocess the uploaded image
        input_image = preprocess_image(image_name, img_size=img_size)

        if input_image is None:
            print(f"Failed to process the image: {image_name}")
            continue

        # Generate random noise
        noise = tf.random.normal([1, 100])





        # Display the original and generated images
        plt.figure(figsize=(10, 5))

        # Original image
        plt.subplot(1, 2, 1)
        plt.title("Uploaded Image")
        plt.imshow(np.asarray(Image.open(image_name).resize((img_size, img_size))))
        plt.axis('off')

        # Generated angry face
        plt.subplot(1, 2, 2)
        plt.title("Generated Angry Face")
        plt.imshow(generated_image, cmap="gray")  # Assuming GAN outputs grayscale
        plt.axis('off')

        plt.show()

# Run the test function
test_gan_model(generator)


Please upload a face image (any format like .jpg, .png, etc.)


Saving Testimages004.jpeg to Testimages004.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step


IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Verify connection by listing the contents of your drive
!ls /content/drive/MyDrive
